In [519]:
import pandas as pd
import numpy as np
import duckdb as db
import skimpy as sk

In [520]:
"""Issues Faced 
Loading Dataset (Unicode Error)
Data Capture Error
(Values stored in Customer_ID)
"""

'Issues Faced \nLoading Dataset (Unicode Error)\nData Capture Error\n(Values stored in Customer_ID)\n'

In [ ]:
data = pd.read_csv("E:\\Projects 25\\Student-Records\\Unclean_Dataset_1.csv", encoding='ISO-8859-1')

### Normalize Column Names

In [522]:
data.columns = [x.strip().lower() for x in data.columns]

To ensure consistency and avoid issues caused by extra spaces or inconsistent casing in column headers, we normalize all column names by:
- **Stripping** leading and trailing spaces
- **Converting** all names to lowercase

This makes it easier to work with the dataset programmatically, as column names become predictable and uniform.

### Preview the Data

After normalizing the column names, it’s important to inspect the first few rows of the dataset.  
Using `.head()` provides a quick snapshot of the structure, allowing us to:

- Verify that column normalization was applied correctly  
- Understand the data types and sample values  
- Identify potential data quality issues (e.g., missing values, formatting inconsistencies)

Problem 1 
Data is trapped in the Student_ID column 
Solution 
Use the split_part function from sqllite to split the values across the columns 

In [524]:
data.columns

Index(['student_id', 'first_name', 'last_name', 'age', 'gender', 'course',
       'enrollment_date', 'total_payments'],
      dtype='object')

### Parse DataFrame into DuckDB for SQL Transformations  

In [ ]:
con = db.connect()
con.execute("CREATE OR REPLACE TABLE df AS SELECT * FROM data")

To leverage the power of SQL for data cleaning and transformation, we load the Pandas DataFrame into a **DuckDB in-memory table**.  
This allows us to:  

- Use **SQL queries** directly on the DataFrame  
- Simplify complex transformations with SQL syntax  
- Maintain a reproducible and efficient data pipeline  

The following command creates (or replaces) a table named `df` inside DuckDB, containing all rows from the Pandas DataFrame `data`.  

### Preview the Data

In [527]:
con.execute("Select * from df").df()

,student_id,first_name,last_name,age,gender,course,enrollment_date,total_payments
0,101 | John | Smith | 22 | M ...,None,None,None,None,None,None,None
1,102 | Emily | Johnson | 24 | F ...,None,None,None,None,None,None,None
2,103 | Michael | Williams | 21 | M ...,None,None,None,None,None,None,None
3,104 | Sarah | Brown | 23 | F ...,None,None,None,None,None,None,None
4,105 | David | Davis | 20 | M ...,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
131,232 | Amina | Adekunle | 21 | ...,None,None,None,None,None,None,None
132,234 | Ibrahim | Okafor | 22 | M...,None,None,None,None,None,None,None
133,235 | Zainab | Abiodun | 23 | ...,None,None,None,None,None,None,None
134,236 | Nasiru | Adebayo | 24 | ...,None,None,None,None,None,None,None


After normalizing the column names, it’s important to inspect the first few rows of the dataset.  
Using `SQL Based Logic` provides a quick snapshot of the structure, allowing us to:

- Verify that column normalization was applied correctly  
- Understand the data types and sample values  
- Identify potential data quality issues (e.g., missing values, formatting inconsistencies)

### Transformation 1: Extracting and Populating Data from `student_id`

In [528]:
con.execute("""
UPDATE df
SET
    student_id = SPLIT_PART(student_id, '|', 1),
    first_name = CASE 
                    WHEN first_name IS NULL AND SPLIT_PART(student_id, '|', 2) IS NOT NULL 
                    THEN SPLIT_PART(student_id, '|', 2)
                    ELSE first_name
                 END,
    last_name = CASE 
                    WHEN last_name IS NULL AND SPLIT_PART(student_id, '|', 3) IS NOT NULL 
                    THEN SPLIT_PART(student_id, '|', 3)
                    ELSE last_name
                 END,
    age = CASE 
             WHEN age IS NULL AND SPLIT_PART(student_id, '|', 4) IS NOT NULL 
             THEN SPLIT_PART(student_id, '|', 4)
             ELSE age
          END,
    gender = CASE 
                WHEN gender IS NULL AND SPLIT_PART(student_id, '|', 5) IS NOT NULL 
                THEN SPLIT_PART(student_id, '|', 5)
                ELSE gender
             END,
    course = CASE 
                WHEN course IS NULL AND SPLIT_PART(student_id, '|', 6) IS NOT NULL 
                THEN SPLIT_PART(student_id, '|', 6)
                ELSE course
             END,
    enrollment_date = CASE 
                        WHEN enrollment_date IS NULL AND SPLIT_PART(student_id, '|', 7) IS NOT NULL 
                        THEN SPLIT_PART(student_id, '|', 7)
                        ELSE enrollment_date
                     END,
    total_payments = CASE 
                        WHEN total_payments IS NULL AND SPLIT_PART(student_id, '|', 8) IS NOT NULL 
                        THEN SPLIT_PART(student_id, '|', 8)
                        ELSE total_payments
                     END
WHERE 
    student_id IS NOT NULL
    AND student_id <> ''  -- Use <> '' to check for non-empty string instead of LENGTH
    AND (
        first_name IS NULL OR last_name IS NULL OR age IS NULL
        OR gender IS NULL OR course IS NULL OR enrollment_date IS NULL 
        OR total_payments IS NULL
    )
""")


In the raw dataset, multiple attributes (e.g., name, age, gender, course, enrollment date, payments) were **embedded inside the `student_id` column** using the `|` delimiter.  
This design caused issues such as:  

- Missing values in key columns (`first_name`, `last_name`, `age`, etc.)  
- Difficulty in performing queries or transformations  
- Poor normalization and data readability  

To resolve this, we applied an **SQL transformation in DuckDB** that:  
1. Splits the `student_id` column using `SPLIT_PART` based on the `|` delimiter.  
2. Populates missing fields (`first_name`, `last_name`, `age`, `gender`, `course`, `enrollment_date`, `total_payments`) with the corresponding extracted values.  
3. Cleans the `student_id` column to only keep the unique identifier (the first split part).  

This ensures the dataset is structured properly and avoids redundancy in storing values inside a single column.  

### Transformation 2: Standardizing Missing Enrollment Dates  

In [529]:
con.execute("""
UPDATE df
SET enrollment_date = NULL
WHERE enrollment_date = 'NA';
""")

During inspection, we noticed that the `enrollment_date` column contained the string `"NA"` instead of proper null values.  
This creates problems because:  

- `"NA"` is treated as a **string**, not a missing value  
- It prevents correct **date parsing and transformations**  
- It can lead to inaccurate results during analysis (e.g., when filtering by date ranges)  

To address this, we replaced all `"NA"` values in `enrollment_date` with **SQL `NULL`**, ensuring missing values are consistently represented.  
This standardization makes future transformations (such as casting to `DATE` or aggregating by enrollment period) more effective and reliable.  

### Standardize `gender` Casing and Trim Whitespace

In the dataset, we observed issues where:  
- `gender` and `age` were sometimes **interchanged**  
- `gender` contained both **age and gender** (e.g., `"M|23"`)  
- Inconsistent casing (`male`, `Female`, `f`, `M`, etc.)  

As a first step, we standardized the `gender` column by:  
- Removing extra whitespace  
- Converting all values to uppercase  

This ensures uniformity before handling more complex issues such as swapped or combined values.  

### Transformation 4: Trimming Spaces in `course`  

In [530]:
con.execute("""
UPDATE df
SET gender = UPPER(gender)
WHERE gender IS NOT NULL;
""")

In [532]:
con.query("Select distinct course from df order by course asc").df()

,course
0,
1,4
2,Cyber Security
3,Data Analysis
4,Data Analytics
5,Data Science
6,Machine Learnin
7,Machine Learning
8,Web Developmen
9,Web Development


### Transformation 4: Cleaning and Standardizing `course`  

During inspection, we identified inconsistencies in the `course` column such as:  
- Leading or trailing spaces (`"Data Science "` vs `"Data Science"`)  
- Slight variations in spelling or truncation (`"Machine Learning"` vs `"Machine Learnin"`)  

These inconsistencies reduce data quality and can cause issues during analysis, such as:  
- Duplicate categories when grouping by course  
- Difficulty in filtering or joining with reference data  

As a first step, we removed leading and trailing spaces using the `TRIM()` function.  
This prepares the column for deeper standardization in later steps, such as correcting misspellings and mapping variations to a canonical set of course names.  


In [531]:
con.execute("Update df set course = trim(course)")

### Transformation 5: Identifying and Resolving Spelling Inconsistencies in `course`  

After trimming spaces, we still observed **spelling inconsistencies and variations** in the `course` column.  
Examples include:  
- `"Machine Learnin"` vs `"Machine Learning"`  
- Variations of `"Web Development"` (e.g., `"Web Develpment"`, `"Web Developmen"`, `"Web Developmet"`)  
- Overlaps between `"Data Analysis"` and `"Data Analytics"`  

Such inconsistencies create fragmented categories, making it difficult to analyze course enrollment accurately.  

To address this, we standardized the values by:  
- Replacing empty strings (`''`) with `NULL`  
- Mapping common misspellings and variations to a canonical label  
- Ensuring consistency across related course names  

This transformation improves **data quality** and ensures that groupings and aggregations on `course` will be meaningful.  


In [533]:
con.execute("""UPDATE df
SET course = CASE 
                WHEN course = '' THEN NULL
                WHEN course IN ('Machine Learnin', 'Machine Learning') THEN 'Machine Learning'
                WHEN course LIKE 'Web Develpment%' OR course LIKE 'Web Developmen%' OR course LIKE 'Web Development%' OR course LIKE 'Web Developmet%' THEN 'Web Development'
                WHEN course LIKE 'Data Analysis%' OR course = 'Data Analytics' THEN 'Data Analytics'
                ELSE course
             END
""")

### Transformation 6: Handling Invalid `student_id` Entries  

During data profiling, we discovered instances where the `student_id` column incorrectly contained the **student's first name** instead of a proper identifier.  
These cases indicate missing or invalid student IDs and could cause issues in:  
- Uniquely identifying students  
- Joining with other datasets  
- Aggregations and analysis  

To address this, we replaced these invalid entries with the placeholder `'Unknown'`, making it clear that the student ID is missing or incorrect.  
This approach preserves data integrity while allowing further transformations and analysis.  

In [534]:
con.execute("""
UPDATE df
SET student_id = 'Unknown'
WHERE student_id = first_name;
""")

### Transformation 7: Adding `currency` Column  

The `total_payments` column contains **varying currencies**, which can cause confusion when analyzing or aggregating payment data.  
To handle this, we added a new column called `currency` to explicitly capture the type of currency for each payment.  

This prepares the dataset for:  
- Standardizing or converting amounts into a single currency  
- Performing accurate financial analyses  
- Avoiding misinterpretation of payment values  

In [535]:
con.execute("ALTER Table df add column currency varchar")

### Transformation 8: Classifying Payment Currency  

The `total_payments` column contains various currency symbols (e.g., `$`, `£`) and ambiguous entries (`?`), which makes financial analysis inconsistent.  

To improve clarity and standardize the data, we classified the currency type by inspecting the `total_payments` values:  
- `$` → `'USD'`  
- `£` → `'Euro'`  
- `?` or unmatched → `NULL`  

This transformation ensures that each payment has a clearly identified currency, enabling accurate aggregations, conversions, and reporting.  


In [536]:
con.execute("""
UPDATE df
SET currency = CASE
    WHEN total_payments LIKE '%$%' THEN 'USD'    -- If total_payments starts with '$', set currency to 'USD'
    WHEN total_payments LIKE '%£%' THEN 'Euro'   -- If total_payments starts with '£', set currency to 'Euro'
    WHEN total_payments LIKE  '%?%' THEN NULL         -- If total_payments is '?', set currency to NULL
    ELSE NULL                                   -- If no match, set currency to NULL
END
""")

### Transformation 9: Cleaning `age` Column  

During data inspection, we found that some entries in the `age` column contained the `'*'` character, which can occur due to formatting errors or data entry issues.  

To ensure that `age` values are numeric and consistent:  
- We replaced all `'*'` characters with an empty string (`''`)  
- This prepares the column for type conversion and further analysis  

Cleaning these anomalies is crucial for accurate age-related calculations and validation.  
python
Copy code


In [537]:
con.execute("UPDATE df SET age = REPLACE(age, '*', '')")

### Transformation 10: Converting Empty `age` Values to NULL  

After removing invalid characters (like `'*'`), some entries in the `age` column were left as empty strings (`''`).  
Empty strings do not behave like proper missing values in SQL and can interfere with:  
- Aggregations and calculations  
- Filtering or grouping by age  
- Data quality checks  

To address this, we replaced all empty string entries in `age` with `NULL`.  
This ensures that missing ages are properly recognized and handled in later transformations.  

In [538]:
con.execute("Update df set age = NULL where age = ''   ")

### Transformation 11: Resolving Complex Issues in `age` and `gender`  

During data cleaning, we observed several complex inconsistencies in the `age` and `gender` columns:  
- `age` and `gender` values were sometimes **swapped** (e.g., age = `'M'` and gender = actual age).  
- `gender` occasionally contained **both age and gender** in one string (e.g., `'M23'`).  

To resolve these issues, we applied a **long `CASE` statement**:  
1. **Swapped values:** If `age` contains `'M'` or `'F'` and `gender` does not, we swap the values to correct the assignment.  
2. **Split combined values:** If `age` is `NULL` and `gender` contains more than one character, we:  
   - Set `age` = last two characters of `gender`  
   - Set `gender` = first character of `gender`  

This transformation ensures that both `age` and `gender` are separated, correctly assigned, and ready for accurate analysis and further transformations.  


In [539]:
con.execute("""UPDATE df
SET
    age = CASE
        -- Condition 1: When age is 'M' or 'F', and gender is not 'M' or 'F', swap the values
        WHEN age IN ('M', 'F') AND gender NOT IN ('M', 'F') THEN gender
        
        -- Condition 2: When age is NULL and gender length > 1, set age = right(gender, 2) and gender = left(gender, 1)
        WHEN age IS NULL AND LENGTH(gender) > 1 THEN RIGHT(gender, 2)
        
        ELSE age
    END,
    
    gender = CASE
        -- Condition 1: When age is 'M' or 'F', and gender is not 'M' or 'F', swap the values
        WHEN age IN ('M', 'F') AND gender NOT IN ('M', 'F') THEN age
        
        -- Condition 2: When age is NULL and gender length > 1, set age = right(gender, 2) and gender = left(gender, 1)
        WHEN age IS NULL AND LENGTH(gender) > 1 THEN LEFT(gender, 1)
        
        ELSE gender
    END
WHERE age IS NOT NULL OR gender IS NOT NULL;""")

### Transformation 12: Standardizing `enrollment_date` Format  

The `enrollment_date` column contained inconsistent date formats, including:  
- `YYYY-MM-DD`  
- Other unspecified or irregular formats  

Inconsistent date formats can cause:  
- Errors in date-based calculations  
- Difficulties in filtering or grouping by month/year  
- Issues with downstream transformations or visualizations  

To resolve this, we implemented a logic to:  
1. Detect dates in the `YYYY-MM-DD` pattern.  
2. Reformat these dates to the standard `DD-MM-YYYY` format using `STRPTIME` and `STRFTIME`.  
3. Keep other formats as-is for manual inspection or later correction.  

This ensures that all recognized dates follow a consistent format, improving data reliability and analysis readiness.  


In [564]:
con.execute("""WITH CTE AS (
    SELECT * FROM df
)
SELECT 
    enrollment_date,
    SPLIT_PART(enrollment_date, '-', 1) AS year,
    CASE 
        -- Check if the first part of the date is a 4-digit year starting with '20'
        WHEN SPLIT_PART(enrollment_date, '-', 1) LIKE '20__' 
             AND LENGTH(SPLIT_PART(enrollment_date, '-', 1)) = 4
             AND SPLIT_PART(enrollment_date, '-', 2) IS NOT NULL
             AND SPLIT_PART(enrollment_date, '-', 3) IS NOT NULL
            THEN 'Year-Month-Day'  -- Format: YYYY-MM-DD
        ELSE 'Other Format'  -- Any other format
    END AS Format_Check,
    
    -- Reformat the date to DD-MM-YYYY if it matches the YYYY-MM-DD pattern
    CASE 
        WHEN enrollment_date LIKE '20__-%' THEN 
            STRFTIME(STRPTIME(TRIM(enrollment_date), '%Y-%m-%d'), '%d-%m-%Y')
        ELSE  enrollment_date-- Keep it as-is if it doesn't match
    END AS Reformatted_Date

FROM CTE;""").df()

,enrollment_date,year,Format_Check,Reformatted_Date
0,2022-05-15,2022,Other Format,2022-05-15
1,2022-03-18,2022,Other Format,2022-03-18
2,2022-06-25,2022,Other Format,2022-06-25
3,None,None,Other Format,None
4,2021-09-10,2021,Other Format,2021-09-10
...,...,...,...,...
131,2022-08-12,2022,Other Format,2022-08-12
132,2022-02-03,2022,Other Format,2022-02-03
133,2022-07-21,2022,Other Format,2022-07-21
134,2022-05-09,2022,Other Format,2022-05-09


### Final Step: Save Cleaned Dataset  

After performing all data cleaning and transformations, we save the final dataset to a CSV file for:  
- Reproducibility and sharing  
- Further analysis or reporting  
- Ensuring the cleaned version is preserved separate from raw data  

The cleaned dataset will be exported to the project folder:

``E:\Projects 25\Student-Records``  


In [ ]:
# Fetch the cleaned data from DuckDB into a Pandas DataFrame
cleaned_df = con.execute("SELECT * FROM df").df()

# Save the cleaned DataFrame to CSV in the project folder
output_path = r"E:\Projects 25\Student-Records\cleaned_student_records.csv"
cleaned_df.to_csv(output_path, index=False)

print(f"Cleaned dataset successfully saved to: {output_path}")
